# Récupération des transactions BTCUSDT par minute sur Binance
Ce notebook télécharge et agrège les transactions BTCUSDT par minute sur 1 an et demi.

In [ ]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pyperclip

# Copiez la clé API Binance dans KeePassXC puis exécutez cette cellule
api_key = pyperclip.paste()

# Copiez le secret API Binance dans KeePassXC puis exécutez la cellule suivante
# (recommandé : séparez la saisie en deux cellules pour plus de sécurité)

# Dans la cellule suivante :
# import pyperclip
# api_secret = pyperclip.paste()
# client = Client(api_key, api_secret)


Veuillez copier la clé API Binance dans le presse-papier, puis appuyez sur Entrée.


In [ ]:
# Définir la période
end_date = datetime.now()
start_date = end_date - timedelta(days=547)  # 1 an et demi

interval = Client.KLINE_INTERVAL_1MINUTE
symbol = 'BTCUSDT'

# Binance limite à 1000 klines par requête, soit ~16h40
all_klines = []
current = start_date

while current < end_date:
    next_time = current + timedelta(minutes=1000)
    klines = client.get_klines(
        symbol=symbol,
        interval=interval,
        startTime=int(current.timestamp() * 1000),
        endTime=int(next_time.timestamp() * 1000)
    )
    all_klines.extend(klines)
    current = next_time
    if len(klines) < 1000:
        break  # Fin des données

# Convertir en DataFrame
df = pd.DataFrame(all_klines, columns=[
    'open_time', 'open', 'high', 'low', 'close', 'volume',
    'close_time', 'quote_asset_volume', 'number_of_trades',
    'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
])
df['minute'] = pd.to_datetime(df['open_time'], unit='ms')
df.set_index('minute', inplace=True)
df = df[['open', 'high', 'low', 'close', 'volume']]
df = df.astype(float)
df.head()